# Zillow Challenge

##  Data input

In [104]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import seterr,isneginf,array
from datetime import datetime
from pandas import compat
from operator import itemgetter
from sklearn import tree
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.neighbors import KDTree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from IPython.display import Image

import seaborn as sns

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 200)
pd.options.display.float_format = '{:20,.2f}'.format
compat.PY3 = False

def Decision_Tree_Image(decision_tree, feature_names, name="temp"):
    
    # Etrainport our decision tree to graphviz format
    dot_file = tree.export_graphviz(decision_tree, out_file='images/' + name + '.dot', feature_names=feature_names)
    
    # Call graphviz to make an image file from our decision tree
    os.system("dot -T png images/" + name + ".dot -o images/" + name + ".png")
    
    # Return the .png image so we can see it
    return Image(filename='images/' + name + '.png')

In [105]:
train_f = "train_2016_v2.csv"
train = pd.read_csv(train_f, index_col = 'parcelid', parse_dates=['transactiondate'])
train['transactiondate'] = pd.to_datetime(train['transactiondate']).astype(int)

properties_f = "properties_2016.csv"
properties = pd.read_csv(properties_f, index_col = 'parcelid')

train_bak = train.join(properties) 

In [106]:
#### TEST MODE  #####
def getdata(pcnt): 
    return train_bak.sample(frac=pcnt)

##  Feature creation

In [107]:
features_base = set()
features_created_custom = set()
features_created_null = set()

In [108]:
#################
#  _isnull
#################
def add_isnull(dfs,feature):
    featurename = feature+"_isnull"
    features_created_null.add(featurename)
    dfs[0][featurename] = dfs[0][feature].isnull()

In [109]:
#################
#  taxrate
#################
def add_taxrate(dfs):
    features_created_custom.add('taxrate')
    dfs[0]['taxrate'] = np.divide(dfs[0]['taxamount'],dfs[0]['taxvaluedollarcnt'])

In [110]:
#################
#  roomcntdiff
#################
def add_roomcntdiff(dfs):
    features_created_custom.add('roomcntdiff')
    dfs[0]['roomcntdiff'] = np.where(dfs[0]['roomcnt']==0, 
                                    np.negative(dfs[0]['bedroomcnt'].add(dfs[0]['bathroomcnt'])),
                                    dfs[0]['roomcnt']
                            )

In [111]:
#################
# add_transactioncnt
#################

def add_transactioncnt(dfs):
    features_created_custom.add('transactioncnt')
    n1 = 200  # square ft bucketing
    monthlag = 3
    cols = ['transactionmonth', 'regionidzip', 'calculatedbathnbr']

    dfs[0] = dfs[0].drop(['transactioncnt'], errors='ignore', axis=1)

    dfs[0]['transactionmonth']    = pd.to_datetime(dfs[0]['transactiondate']).dt.month
    dfs[0]['transactionmonthlag'] = np.subtract(dfs[0]['transactionmonth'],monthlag)    
    dfs[0]['calculatedfinishedsquarefeet_t'] = np.ceil(np.divide(dfs[0]['calculatedfinishedsquarefeet'], n1))

    sumdf = pd.DataFrame({'transactioncnt' : dfs[0].groupby([
                    'transactionmonth' 
                    ,'regionidzip' 
                    ,'roomcnt'
                    ,'calculatedfinishedsquarefeet_t'
                ]).size()}).reset_index()
    sumdf.rename(columns={'transactionmonth': 'transactionmonthlag'}, inplace=True)
    dfs[0] = pd.merge(dfs[0], sumdf, on=['transactionmonthlag' 
                                ,'regionidzip' 
                                ,'roomcnt'
                                 ,'calculatedfinishedsquarefeet_t'
                                ], how='left')    
    dfs[0] = dfs[0][dfs[0]['transactionmonthlag']>0]  # Drop sales done in earlier than monthlag months
    dfs[0] = dfs[0].drop(['transactionmonth','transactionmonthlag','calculatedfinishedsquarefeet_t'], axis=1)

In [112]:
#################
#  distanceavg
#################
def add_distanceavg(dfs):  
    # Convert to miles, 1 degree = 69 miles
    #df['latitudemi']  = df['latitude']* 5280 * 69/1e6
    #df['longitudemi'] *= df['longitude'] * 5280* 69/1e6

    features_created_custom.add('distanceavg')

    n1 = 500  # square ft   # TODO: exponentiate as it grows bigger
    n2 = 20000 # lot size
    dfs[0]['calculatedfinishedsquarefeet_t'] = np.ceil(np.divide(dfs[0]['calculatedfinishedsquarefeet'], n1))
    dfs[0]['lotsizesquarefeet_t'] = np.ceil(np.divide(dfs[0]['lotsizesquarefeet'], n2))    

    monthlag = 3
    dfs[0]['transactionmonth']    = pd.to_datetime(dfs[0]['transactiondate']).dt.month
    dfs[0]['transactionmonthlag'] = np.subtract(dfs[0]['transactionmonth'],monthlag)    
   
    n=3    # n-1 nearest neighbor
    dfs[0]['distanceavg'] = np.nan     # Unknown distance, one of feastures not available, assume 20
    distancefeatures = ['latitude', 'longitude']   # features to measure distance
    for i in dfs[0]['calculatedbathnbr'].unique():         # features to define comparables
        for k in dfs[0]['lotsizesquarefeet_t'].unique():   
            for l in dfs[0]['transactionmonthlag'].unique():
                if ((~np.isnan(i)) 
                        #& (~np.isnan(j)) 
                        & (~np.isnan(k)) 
                        & (~np.isnan(l)) 
                       ):
                    mask1 = (
                            (dfs[0]['calculatedbathnbr'] == i)   
                          #  & (df['bathroomcnt'] == j) 
                            & (dfs[0]['lotsizesquarefeet_t'] == k) 
                            & (dfs[0]['transactionmonth'] == l)
                           ) 
                    df_subset1 = dfs[0][mask1]
                    
                    mask2 = (
                            (dfs[0]['calculatedbathnbr'] == i)   
                          #  & (df['bathroomcnt'] == j) 
                            & (dfs[0]['lotsizesquarefeet_t'] == k) 
                            & (dfs[0]['transactionmonthlag'] == l)
                           ) 
                    df_subset2 = dfs[0][mask2]
                    
                    if ((len(df_subset1))>n-1 &
                         (len(df_subset2))>1):
                        kdt = KDTree(df_subset1.loc[:,distancefeatures], 
                                     leaf_size=30, 
                                     metric='euclidean')
                        dists, neighs=kdt.query(df_subset2.loc[:,distancefeatures], 
                                                k=n, 
                                                return_distance=True) 
                        distanceavg=dists.mean(axis=1)
                        #res = np.log(distanceavg)
                        #res[isneginf(res)]=0
                        dfs[0].loc[mask2,'distanceavg'] = distanceavg
    dfs[0] = dfs[0].drop(['calculatedfinishedsquarefeet_t','lotsizesquarefeet_t',
                         'transactionmonth','transactionmonthlag'], axis=1)
    
#['distanceavgmi']=train['distanceavg']* 69/1e6

In [113]:
def createfeatures(df, features):
    functions = {
                'taxrate': add_taxrate,
                'roomcntdiff':add_roomcntdiff,
                'transactioncnt':add_transactioncnt,
                'distanceavg':add_distanceavg   
                } 
    for f in features:
        functions[f]([df])

In [114]:
def createfeaturesnull(df):
    for f in (train.columns - ['logerror']):
        add_isnull([train],f)

In [115]:
train = getdata(1)

In [116]:
# Add null features
createfeaturesnull(train)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  from ipykernel import kernelapp as app


In [117]:
# Add custom features
createfeatures(train,[
               'taxrate',
               'roomcntdiff',
               'transactioncnt',
               'distanceavg'
              ])

In [118]:
#Binary
features_binary = {'hashottuborspa',
                'pooltypeid10',
                'pooltypeid2',
                'pooltypeid7',
                'fireplaceflag',
                'taxdelinquencyflag'}

for f in features_binary:
    train[f] = pd.Categorical(train[f]).codes

In [119]:
# Types
features_types = {'airconditioningtypeid',
                  'architecturalstyletypeid',
                  'buildingqualitytypeid',
                  'buildingclasstypeid',
                  'decktypeid',
                  'heatingorsystemtypeid',
                  'propertycountylandusecode',
                  'propertylandusetypeid',
                  'propertyzoningdesc',
                  'typeconstructiontypeid',
                  'storytypeid'}

for f in features_types:
    train[f] = pd.Categorical(train[f]).codes

In [120]:
train = train.fillna(-1)

In [121]:
train_full = train

##  Feature selection

In [122]:
features_created_null = set([
'airconditioningtypeid_isnull',
 'airconditioningtypeid_isnull_isnull',
 'architecturalstyletypeid_isnull',
 'architecturalstyletypeid_isnull_isnull',
 'assessmentyear_isnull',
 'assessmentyear_isnull_isnull',
 'basementsqft_isnull',
 'basementsqft_isnull_isnull',
 'bathroomcnt_isnull',
 'bathroomcnt_isnull_isnull',
 'bedroomcnt_isnull',
 'bedroomcnt_isnull_isnull',
 'buildingclasstypeid_isnull',
 'buildingclasstypeid_isnull_isnull',
 'buildingqualitytypeid_isnull',
 'buildingqualitytypeid_isnull_isnull',
 'calculatedbathnbr_isnull',
 'calculatedbathnbr_isnull_isnull',
 'calculatedfinishedsquarefeet_isnull',
 'calculatedfinishedsquarefeet_isnull_isnull',
 'censustractandblock_isnull',
 'censustractandblock_isnull_isnull',
 'decktypeid_isnull',
 'decktypeid_isnull_isnull',
 'finishedfloor1squarefeet_isnull',
 'finishedfloor1squarefeet_isnull_isnull',
 'finishedsquarefeet12_isnull',
 'finishedsquarefeet12_isnull_isnull',
 'finishedsquarefeet13_isnull',
 'finishedsquarefeet13_isnull_isnull',
 'finishedsquarefeet15_isnull',
 'finishedsquarefeet15_isnull_isnull',
 'finishedsquarefeet50_isnull',
 'finishedsquarefeet50_isnull_isnull',
 'finishedsquarefeet6_isnull',
 'finishedsquarefeet6_isnull_isnull',
 'fips_isnull',
 'fips_isnull_isnull',
 'fireplacecnt_isnull',
 'fireplacecnt_isnull_isnull',
 'fireplaceflag_isnull',
 'fireplaceflag_isnull_isnull',
 'fullbathcnt_isnull',
 'fullbathcnt_isnull_isnull',
 'garagecarcnt_isnull',
 'garagecarcnt_isnull_isnull',
 'garagetotalsqft_isnull',
 'garagetotalsqft_isnull_isnull',
 'hashottuborspa_isnull',
 'hashottuborspa_isnull_isnull',
 'heatingorsystemtypeid_isnull',
 'heatingorsystemtypeid_isnull_isnull',
 'landtaxvaluedollarcnt_isnull',
 'landtaxvaluedollarcnt_isnull_isnull',
 'latitude_isnull',
 'latitude_isnull_isnull',
 'logerror_isnull_isnull',
 'longitude_isnull',
 'longitude_isnull_isnull',
 'lotsizesquarefeet_isnull',
 'lotsizesquarefeet_isnull_isnull',
 'numberofstories_isnull',
 'numberofstories_isnull_isnull',
 'poolcnt_isnull',
 'poolcnt_isnull_isnull',
 'poolsizesum_isnull',
 'poolsizesum_isnull_isnull',
 'pooltypeid10_isnull',
 'pooltypeid10_isnull_isnull',
 'pooltypeid2_isnull',
 'pooltypeid2_isnull_isnull',
 'pooltypeid7_isnull',
 'pooltypeid7_isnull_isnull',
 'propertycountylandusecode_isnull',
 'propertycountylandusecode_isnull_isnull',
 'propertylandusetypeid_isnull',
 'propertylandusetypeid_isnull_isnull',
 'propertyzoningdesc_isnull',
 'propertyzoningdesc_isnull_isnull',
 'rawcensustractandblock_isnull',
 'rawcensustractandblock_isnull_isnull',
 'regionidcity_isnull',
 'regionidcity_isnull_isnull',
 'regionidcounty_isnull',
 'regionidcounty_isnull_isnull',
 'regionidneighborhood_isnull',
 'regionidneighborhood_isnull_isnull',
 'regionidzip_isnull',
 'regionidzip_isnull_isnull',
 'roomcnt_isnull',
 'roomcnt_isnull_isnull',
 'storytypeid_isnull',
 'storytypeid_isnull_isnull',
 'structuretaxvaluedollarcnt_isnull',
 'structuretaxvaluedollarcnt_isnull_isnull',
 'taxamount_isnull',
 'taxamount_isnull_isnull',
 'taxdelinquencyflag_isnull',
 'taxdelinquencyflag_isnull_isnull',
 'taxdelinquencyyear_isnull',
 'taxdelinquencyyear_isnull_isnull',
 'taxvaluedollarcnt_isnull',
 'taxvaluedollarcnt_isnull_isnull',
 'threequarterbathnbr_isnull',
 'threequarterbathnbr_isnull_isnull',
 'transactiondate_isnull',
 'transactiondate_isnull_isnull',
 'typeconstructiontypeid_isnull',
 'typeconstructiontypeid_isnull_isnull',
 'unitcnt_isnull',
 'unitcnt_isnull_isnull',
 'yardbuildingsqft17_isnull',
 'yardbuildingsqft17_isnull_isnull',
 'yardbuildingsqft26_isnull',
 'yardbuildingsqft26_isnull_isnull',
 'yearbuilt_isnull',
 'yearbuilt_isnull_isnull'        
    ])

In [123]:
features_created_custom = set([
                'taxrate',
               'roomcntdiff',
               'transactioncnt',
               'distanceavg'
    ])

In [124]:
features_base = set([
 ##########
 #  train data
 ##########
 'logerror', # Predictor variable 
         
 ##########
 #  Numeric attributes
 ##########
 # ROOMS
 'roomcnt',	#  Total number of rooms in the principal residence

 # BATHROOMS
 'bathroomcnt',	#  Number of bathrooms in home including fractional bathrooms
 'calculatedbathnbr',	#  Number of bathrooms in home including fractional bathroom
 'threequarterbathnbr',	#  Number of 3/4 bathrooms in house (shower + sink + toilet)
 'fullbathcnt',	#  Number of full bathrooms (sink, shower + bathtub, and toilet) present in home

 # BEDROOMS
 'bedroomcnt',	#  Number of bedrooms in home 
        
  # MEASUREMENTS
  'basementsqft', #  Finished living area below or partially below ground level
  'calculatedfinishedsquarefeet',	#  Calculated total finished living area of the home 
  'finishedfloor1squarefeet',	#  Size of the finished living area on the first (entry) floor of the home
  'finishedsquarefeet6',	# Base unfinished and finished area
  'finishedsquarefeet12',	# Finished living area
  'finishedsquarefeet13',	# Perimeter  living area
  'finishedsquarefeet15',	# Total area
  'finishedsquarefeet50',	#  Size of the finished living area on the first (entry) floor of the home
  'lotsizesquarefeet',	#  Area of the lot in square feet
        
 # FIREPLACE
 'fireplacecnt',	#  Number of fireplaces in a home (if any)

 # GARAGE
 'garagecarcnt',	#  Total number of garages on the lot including an attached garage
 'garagetotalsqft',	#  Total number of square feet of all garages on lot including an attached garage

 # TAXES
 'landtaxvaluedollarcnt',	# The assessed value of the land area of the parcel
 'structuretaxvaluedollarcnt',	# The assessed value of the built structure on the parcel
 'taxamount',	# The total property tax assessed for that assessment year
 'taxvaluedollarcnt',	# The total tax assessed value of the parcel

 # POOL
 'poolcnt',	#  Number of pools on the lot (if any)
 'poolsizesum',	#  Total square footage of all pools on property

 # YARD
 'yardbuildingsqft17',	# Patio in  yard
 'yardbuildingsqft26',	# Storage shed/building in yard

 # BUILDING
 'numberofstories',	#  Number of stories or levels the home has
 'unitcnt',	#  Number of units the structure is built into (i.e. 2 = dupletrain, 3 = tripletrain, etc...)

 ##########
 #  Time
 ##########
#####    'transactiondate',   ## from train data
 'yearbuilt',	#  The Year the principal residence was built      
 'taxdelinquencyyear',	# Year for which the unpaid propert tatraines were due 
 'assessmentyear',	# The year of the property tax assessment 
     
 ##########
 #  Location
 ########## 
 'latitude',	#  Latitude of the middle of the parcel multiplied by 10e6
 'longitude',	#  Longitude of the middle of the parcel multiplied by 10e6        
 'regionidcounty',	# County in which the property is located
 'regionidcity',	#  City in which the property is located (if any)
 'regionidzip',	#  Zip code in which the property is located
 'regionidneighborhood',	# Neighborhood in which the property is located
 'rawcensustractandblock',	#  Census tract and block ID combined - also contains blockgroup assignment by etraintension
 'censustractandblock',	#  Census tract and block ID combined - also contains blockgroup assignment by etraintension        
        
 ##########
 #  Binary
 ##########         
 'hashottuborspa',	#  Does the home have a hot tub or spa - array([nan, True], dtype=object)
        # array([nan, True], dtype=object)
 'pooltypeid10',	#  Spa or Hot Tub - [array([ nan,   1.])]
        # array([ nan,   1.])
 'pooltypeid2',	#  Pool with Spa/Hot Tub - [array([ nan,   1.])]
        # array([ nan,   1.])
 'pooltypeid7',	#  Pool without hot tub - [array([ nan,   1.])]
        # array([ nan,   1.])
 'fireplaceflag',	#  Is a fireplace present in this home - array([nan, True], dtype=object)
        # array([nan, True], dtype=object)
 'taxdelinquencyflag',	# Property tatraines for this parcel are past due as of 2015 - array([nan, 'Y'], dtype=object)
        # array([nan, 'Y'], dtype=object)
        
 ##########
 #  Types
 ##########    
 'airconditioningtypeid', #  Type of cooling system present in the home (if any)
 'architecturalstyletypeid', #  Architectural style of the home (i.e. ranch, colonial, split-level, etc…) 
        # array([ nan,   7.,  21.,   8.,   2.,   3.,   5.,  10.,  27.])
 'buildingqualitytypeid',	#  Overall assessment of condition of the building from best (lowest) to worst (highest)
        # array([ nan,   7.,   4.,  10.,   1.,  12.,   8.,   3.,   6.,   9.,   5.,
        #11.,   2.])
 'buildingclasstypeid',	# The building framing type (steel frame, wood frame, concrete/brick) 
        # array([ nan,   3.,   4.,   5.,   2.,   1.])
 'decktypeid',	# Type of deck (if any) present on parcel
        # array([ nan,  66.])
 'heatingorsystemtypeid',	#  Type of home heating system
        # array([ nan,   2.,   7.,  20.,   6.,  13.,  18.,  24.,  12.,  10.,   1.,
        # 14.,  21.,  11.,  19.])
 'propertycountylandusecode',	#  County land use code i.e. it's zoning at the county level
        #array([nan,'010D', '0109', '1200', '1210', '010V', '300V', '0100', '0200',
        #'010C', '0300', '1204', '100V', '01HC', '120C', '040V', '1214', ... 
        # ], dtype=object)
 'propertylandusetypeid',	#  Type of land use the property is zoned for
        # array([ 269.,  261.,   47.,   31.,  260.,  266.,  246.,  247.,  248.,
        # 263.,  265.,  275.,  267.,  264.,  270.,   nan])
 'propertyzoningdesc',	#  Description of the allowed land uses (zoning) for that property   
        # array([nan, 'LCA11*', 'LAC2', ..., 'WCR1400000', 'EMPYYY', 'RMM2*'], dtype=object)
 'typeconstructiontypeid',	#  What type of construction material was used to construct the home
        # array([ nan,   6.,   4.,  10.,  13.,  11.])
 'storytypeid',	#  Type of floors in a multi-story house (i.e. basement and main level, split-level, attic, etc.).  See tab for details.
        # array([ nan,   7.])

 ##########
 #  Misc
 ########## 
 #'fips',	#  Federal Information Processing Standard code -  see https://en.wikipedia.org/wiki/FIPS_county_code for more details
        # array([ 6037.,  6059.,  6111.,    nan])
 #'parcelid',	#  Unique identifier for parcels (lots)   
])



In [125]:
features_to_keep = set()
features_to_keep = features_to_keep.union(
                            features_base,
                            features_created_custom,
                            features_created_null
                            )

In [127]:
train_filtered = train_full
for column in train.columns:
    if column not in features_to_keep:
        train_filtered = train_filtered.drop(column, axis=1)

## Sample

In [132]:
def getsample(pcnt): 
    return train_filtered.sample(frac=pcnt)

In [133]:
train_sample = getsample(.1)
Y = train_sample[['logerror']]
X = train_sample.drop('logerror',axis=1)

## Modeling

In [134]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.9)

//anaconda/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [136]:
# Fit regression model
min_samples_split = int(round(len(X_train)/10, 0))
min_samples_leaf = int(round(len(X_train)/200, 0))

print("min_samples_split =", min_samples_split)
print("min_samples_split =", min_samples_leaf)

model = DecisionTreeRegressor(max_depth=None, 
                              min_samples_split = min_samples_split,
                              min_samples_leaf = min_samples_leaf,
                              criterion = "mae")
model.fit(X_train, Y_train)

('min_samples_split =', 451)
('min_samples_split =', 22)


DecisionTreeRegressor(criterion='mae', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=22,
           min_samples_split=451, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [ ]:
# Feature search
from sklearn.feature_selection import RFE
rfe = RFE(model, 6)
fit = rfe.fit(X_train, Y_train)
print("Num Features: %d") % fit.n_features_
print("Selected Features: %s") % fit.support_
print("Feature Ranking: %s") % fit.ranking_

//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [137]:
Y_predict=model.predict(X_test)

In [138]:
# Feature importance
sorted(zip(map(lambda x: round(x, 4), model.feature_importances_), X_train.columns), 
             reverse=True)

[(0.1604, 'taxrate'),
 (0.1463, 'calculatedfinishedsquarefeet'),
 (0.1331, 'lotsizesquarefeet'),
 (0.0714, 'taxvaluedollarcnt'),
 (0.0696, 'landtaxvaluedollarcnt'),
 (0.0677, 'structuretaxvaluedollarcnt'),
 (0.0537, 'taxamount'),
 (0.0458, 'finishedsquarefeet15'),
 (0.0455, 'censustractandblock'),
 (0.0393, 'yearbuilt'),
 (0.0377, 'taxdelinquencyyear'),
 (0.0301, 'longitude'),
 (0.0297, 'hashottuborspa'),
 (0.0275, 'latitude'),
 (0.0262, 'propertyzoningdesc'),
 (0.016, 'heatingorsystemtypeid'),
 (0.0, 'yearbuilt_isnull'),
 (0.0, 'yardbuildingsqft26_isnull'),
 (0.0, 'yardbuildingsqft26'),
 (0.0, 'yardbuildingsqft17_isnull'),
 (0.0, 'yardbuildingsqft17'),
 (0.0, 'unitcnt_isnull'),
 (0.0, 'unitcnt'),
 (0.0, 'typeconstructiontypeid_isnull'),
 (0.0, 'typeconstructiontypeid'),
 (0.0, 'transactiondate_isnull'),
 (0.0, 'threequarterbathnbr_isnull'),
 (0.0, 'threequarterbathnbr'),
 (0.0, 'taxvaluedollarcnt_isnull'),
 (0.0, 'taxdelinquencyyear_isnull'),
 (0.0, 'taxdelinquencyflag_isnull'),
 (0.0

In [139]:
# Tree description
zip(X.columns[model.tree_.feature], 
    model.tree_.threshold, 
    model.tree_.value, 
    model.tree_.n_node_samples,
    model.tree_.children_left, 
    model.tree_.children_right)

[('taxrate', 0.027833841741085052, array([[ 0.006]]), 4514, 1, 52),
 ('calculatedfinishedsquarefeet', 2713.5, array([[ 0.007]]), 4338, 2, 47),
 ('landtaxvaluedollarcnt', 15963.5, array([[ 0.006]]), 3794, 3, 4),
 ('roomcntdiff', -2.0, array([[ 0.0286]]), 47, -1, -1),
 ('yearbuilt', 1919.5, array([[ 0.005]]), 3747, 5, 6),
 ('roomcntdiff', -2.0, array([[-0.0166]]), 108, -1, -1),
 ('longitude', -119223800.0, array([[ 0.006]]), 3639, 7, 8),
 ('roomcntdiff', -2.0, array([[ 0.03295]]), 28, -1, -1),
 ('taxamount', 1774.905029296875, array([[ 0.005]]), 3611, 9, 10),
 ('roomcntdiff', -2.0, array([[ 0.0119]]), 420, -1, -1),
 ('finishedsquarefeet15', 1319.5, array([[ 0.005]]), 3191, 11, 46),
 ('taxrate', 0.017487503588199615, array([[ 0.005]]), 3122, 12, 45),
 ('hashottuborspa', -0.5, array([[ 0.006]]), 2908, 13, 44),
 ('latitude', 34237600.0, array([[ 0.006]]), 2858, 14, 43),
 ('taxdelinquencyyear', 8.5, array([[ 0.007]]), 2456, 15, 42),
 ('taxvaluedollarcnt', 1211644.5, array([[ 0.007]]), 2424, 

In [ ]:
#Decision_Tree_Image(model, X_train.columns)

## Model evaluation

In [141]:
# Zillow MAE (test)
print("MAE: ",metrics.mean_absolute_error(Y_test['logerror'], Y_predict))

('MAE: ', 0.066916216216216212)


In [33]:
print ("Features: ", X.columns)
print ("Tree params: ")
print ("   max_depth ", max_depth)
print ("   min_samples_split ", min_samples_split)
print ("   min_samples_leaf ", min_samples_leaf)
print ("Result: ",metrics.mean_absolute_error(Y_test['logerror'], Y_predict))

('Features: ', Index([u'airconditioningtypeid', u'architecturalstyletypeid', u'basementsqft',
       u'bathroomcnt', u'bedroomcnt', u'buildingclasstypeid',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'decktypeid',
       u'finishedfloor1squarefeet',
       ...
       u'taxvaluedollarcnt_isnull', u'assessmentyear_isnull',
       u'landtaxvaluedollarcnt_isnull', u'taxamount_isnull',
       u'taxdelinquencyflag_isnull', u'taxdelinquencyyear_isnull',
       u'censustractandblock_isnull', u'taxrate', u'roomcntdiff',
       u'distanceavg'],
      dtype='object', length=118))
Tree params: 


NameError: name 'max_depth' is not defined

## Model Execution

In [ ]:
sample_submission_f = '/Users/Jose/Desktop/Zillow/sample_submission.csv'
submission = pd.read_csv(sample_submission_f, index_col='ParcelId')

In [ ]:
#  ATTEMPT 2 - 
logerroravg = train['logerror'].mean()
print("Using logerroravg = ", logerroravg)

In [ ]:
X_all = properties

In [ ]:
# Prediction transaction date
def add_transactiondate(df):
    df['transactiondate'] = pd.datetime(2016,10,1)
    df['transactiondate'] = df['transactiondate'].astype(int)

add_transactiondate(X_all)

In [ ]:
# Add features


In [ ]:
for column in X_all.columns:
    if column not in features_to_keep:
        X_all = X_all.drop(column, axis=1)

In [ ]:
#  Check for missing data
print("Missing: ",X_test.columns.difference( X_all.columns))

In [ ]:
Y_all = model.predict(X_all)

In [ ]:
results = pd.DataFrame(index=X_all.index)
results.index.names = ['ParcelId']
results['201610'] = Y_all
results['201611'] = Y_all
results['201612'] = Y_all
results['201710'] = Y_all
results['201711'] = Y_all
results['201712'] = Y_all

In [ ]:
submission = submission.drop(submission.columns[0:], axis=1)
submission = submission.join(results)

# Use average for properties with missing data
submission.fillna(logerroravg, inplace=True)       
submission.columns

In [ ]:
# Round as per rules
submission = submission.round(4)

## Sanity Checks

In [ ]:
# Average log error
submission.describe().round(4)

In [ ]:
# Check for NaN
submission.isnull().sum()

In [ ]:
# Check if any duplicates
submission[submission.index.duplicated(keep=False)]

In [ ]:
# Check additional values in submission file
submission[~submission.index.isin(properties.index)]

In [ ]:
# Check additional values in properties file
properties[~properties.index.isin(submission.index)]

## Dump File

In [ ]:
# Write file
submission_f = '/Users/Jose/Desktop/Zillow/submission.csv'
submission_fh = open(submission_f, 'wb')
submission.to_csv(submission_fh, sep=',', header='true')
submission_fh.close()

## Historical Records

## Resources

Conclusions:
   * Average logerror = 0.011457219606756575
   * In general, data usually underestimates, with the etrainception of fewer high overestimates.
   * totalroomcnt:  ==0 is a mitrained bag.  != zero is overestimating

In [ ]:
# bedroomcnt
#plt.trainlabel('bedroomcnt')
#plt.ylabel('logerror')
#plt.scatter(train['bathroomcnt'], train['logerror'], 
#           alpha=1, s=2, color='r')
#plt.atrainis([0, 10,-5, 5])
#plt.show()

#date
#plt.trainlabel('transactiondate')
#plt.ylabel('logerror')
#plt.scatter(train['bedroomcnt'], train['logerror'], alpha=1, s=2, color='r')
#plt.atrainis(['2016-01-01', '2017-01-01',-5, 5])
#plt.gcf().autofmt_traindate()
#plt.show()

#import matplotlib as mpl
#plt.trainlabel('longitude')
#plt.ylabel('latitude')
#plt.scatter(train['longitude'], train['latitude'], c=train['logerror'], s=20, lw=0, cmap='seismic')
#plt.atrainis([min(train['longitude']), matrain(train['longitude']),min(train['latitude']), matrain(train['latitude'])])
#plt.colorbar()
#plt.show()

#train2 = train.query('logerror < -2.')

#train2 = train.query('roomcnt == 0 ')

#train2 = train.query('bedroomcnt > 2')

#train2 = train.sample(frac=.10)

#plt.trainlabel('longitude')
#plt.ylabel('latitude')
#plt.scatter(train2['longitude'], train2['latitude'], c=train2['logerror'], s=20, lw=0, cmap='seismic')
#plt.colorbar()
#plt.show()